In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import torchvision
import os
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import time
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
#from models import * #model and dataset
from train import *

if torch.cuda.is_available():  
    torch.cuda.empty_cache()
    device = "cuda:0" 
else:  
    device = "cpu" 
    
print(torch.__version__)

1.8.1+cu101


In [ ]:
torch.manual_seed(16)
imgpath = 'Dataset/train_images'
maskpath = 'Dataset/train_masks'
dataset = Polyp(imgpath,maskpath)
batch = 8

sz = len(dataset)
trlen = int(len(dataset)*0.8)
vllen = sz - trlen
trainset, validset = torch.utils.data.random_split(dataset,[trlen,vllen])

train_loader=DataLoader(trainset,batch,shuffle=True)
valid_loader = DataLoader(validset,batch,shuffle=True)

model = encoder().to(device)
epochs = 15

optimizer = optim.Adadelta(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

for epoch in range(epochs):
    tg = []
    pd = []
    train_accuracy=0.0
    test_accuracy=0.0
    model.train()
    for data, target in tqdm(train_loader):
        data,target=data.to(device),target.to(device)
        optimizer.zero_grad()
        output=model(data.float())
        loss=loss_function(output,target)
        loss.backward()
        optimizer.step()
        _,prediction=torch.max(output.data,1)
        train_accuracy+=int(torch.sum(prediction==target.data))
    train_accuracy=train_accuracy/trlen 
    model.eval()
    
    with torch.no_grad():
        for data, target in tqdm(valid_loader):
            data,target=data.to(device),target.to(device)
            output=model(data.float())
            _,prediction=torch.max(output.data,1)
            test_accuracy+=int(torch.sum(prediction==target.data))
            for k in prediction:
                pd.append(k.item())
            for l in target.data:
                tg.append(l.item())
    torch.save(model,'encoder.pt')
    test_accuracy=test_accuracy/vllen
    print('epoch '+str(epoch+1)+' train acurracy: '+str(train_accuracy)+' test accuracy: '+str(test_accuracy))
    print(confusion_matrix(tg,pd))

100%|██████████| 23/23 [00:05<00:00,  4.30it/s]


epoch 1 train acurracy: 0.35 test accuracy: 0.35555555555555557
[[25  3 26]
 [28  6 32]
 [22  5 33]]


100%|██████████| 23/23 [00:07<00:00,  3.14it/s]


epoch 2 train acurracy: 0.6652777777777777 test accuracy: 0.37222222222222223
[[28  3 23]
 [26  7 33]
 [23  5 32]]


100%|██████████| 23/23 [00:05<00:00,  4.25it/s]


...

100%|██████████| 23/23 [00:06<00:00,  3.44it/s]


epoch 8 train acurracy: 1.0 test accuracy: 0.45555555555555555
[[29 12 13]
 [21 19 26]
 [11 15 34]]


100%|██████████| 23/23 [00:07<00:00,  3.11it/s]


epoch 9 train acurracy: 1.0 test accuracy: 0.43333333333333335
[[32 10 12]
 [26 16 24]
 [17 13 30]]


 23%|██▎       | 21/90 [00:12<00:41,  1.66it/s]

In [ ]:
print(output,target)